In [1]:
# 依赖导入
import pandas as pd
import ijson
import csv
import os
import random

In [2]:
# 定义数据文件路径变量
data3_df_path = "data/trades_3.csv"
data4_df_path = "data/trades_4.csv"
data10_df_path = "data/trades_10.csv"
data13_df_path = "data/trades_13.csv"
data14_df_path = "data/trades_14.csv"
data18_df_path = "data/trades_18.csv"
data_paths = [
    data3_df_path,
    data4_df_path,
    data10_df_path,
    data13_df_path,
    data14_df_path,
    data18_df_path,
]

broker_5_path = "broker_5/1_batch_write_close_2010-01-01_2025-01-01.json"
tradingHistory_path = "login_tradings"

data文件夹中的数据分析：

In [23]:
for path in data_paths:
    print(f"数据文件路径: {path}")
    df = pd.read_csv(path)
    login_count = df['login'].nunique()
    print(f"login的数量有: {login_count}")

数据文件路径: data/trades_3.csv
login的数量有: 11898
数据文件路径: data/trades_4.csv
login的数量有: 1307
数据文件路径: data/trades_10.csv
login的数量有: 11274
数据文件路径: data/trades_13.csv
login的数量有: 4171
数据文件路径: data/trades_14.csv
login的数量有: 5142
数据文件路径: data/trades_18.csv
login的数量有: 11298


broker_5 json数据预处理：

In [3]:
# 定义一个生成器函数，用于逐条读取大型JSON数组文件
def stream_json_array(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        for obj in ijson.items(f, "item"):  # 顶层是数组时，'item' 就是一条记录
            yield obj

In [40]:
allLogins = set()

for record in stream_json_array(
    "broker_5/1_batch_write_close_2010-01-01_2025-01-01.json"
):
    login = str(record.get("login")) + "_" + str(record.get("symbol"))
    login_filename = f"{tradingHistory_path}/{login}.csv"
    
    if login is not None and login not in allLogins:
        allLogins.add(login)
        print(f"新增login: {login}, 当前总数: {len(allLogins)}", end='\r', flush=True)
        # 创建一个新的trades csv文件，文件名为login，保存record中的数据
        with open(login_filename, mode="w", newline='', encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=record.keys())
            writer.writeheader()
            writer.writerow(record)
    else:
        with open(login_filename, mode="a", newline='', encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=record.keys())
            writer.writerow(record)

IncompleteJSONError: parse error: premature EOF
                                       
                     (right here) ------^


In [ ]:
# 删除tradingHistory_path中行数小于等于10的csv文件
for filename in os.listdir(tradingHistory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(tradingHistory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            row_count = sum(1 for _ in f)
        if row_count <= 10:
            os.remove(file_path)
            print(f"已删除: {file_path}", end='\r', flush=True)

In [ ]:
# 完成数据处理后，broker_5中json文件被拆分成多个csv文件，保存在tradingHistory_path目录下, 单个产品交易笔数大于十笔的数量是多少
file_count = len([f for f in os.listdir(tradingHistory_path) if f.endswith('.csv')])
print(f"{tradingHistory_path} 文件夹中的csv文件数量为: {file_count}")

login_tradings 文件夹中的csv文件数量为: 66445


In [8]:
csv_files = [f for f in os.listdir(tradingHistory_path) if f.endswith('.csv')]
if csv_files:
    random_file = random.choice(csv_files)
    random_file_path = os.path.join(tradingHistory_path, random_file)
    print(f"随机抽取的文件: {random_file_path}")
    with open(random_file_path, 'r', encoding='utf-8') as f:
        print(f.read())
else:
    print("目录下没有csv文件。")

随机抽取的文件: login_tradings/778712_USDCAD.csv
broker_id,ticket,login,symbol,cmd,volume,open_time,open_price,sl,tp,close_time,reason,commission,swaps,close_price,profit,server_id
6,38193104,778712,USDCAD,0,1,1600800134,1.33133,1.33,1.335,1600801295,5,0.0,0.0,1.33136,0.02,1
6,38215184,778712,USDCAD,0,1,1600980767,1.333,1.328,1.34,1601029613,5,0.0,-0.02,1.33416,0.87,1
6,38562339,778712,USDCAD,0,1,1603813288,1.31682,0.0,1.33,1603903207,5,0.0,-0.01,1.32773,8.22,1
6,38947716,778712,USDCAD,1,1,1606363347,1.29956,0.0,0.0,1606413916,0,0.0,0.0,1.30217,-2.0,1
6,39327129,778712,USDCAD,0,1,1608320877,1.27955,1.28,1.2975,1608545065,0,0.0,-0.01,1.28827,6.77,1
6,40861860,778712,USDCAD,0,1,1616515861,1.25733,0.0,0.0,1616587326,0,0.0,-0.02,1.25768,0.28,1
6,41657758,778712,USDCAD,1,1,1620363688,1.21474,0.0,0.0,1620364563,0,0.0,0.0,1.21562,-0.72,1
6,41995930,778712,USDCAD,0,1,1622129868,1.20801,1.201,1.2275,1622227081,0,0.0,-0.02,1.20819,0.15,1
6,42283164,778712,USDCAD,0,1,1623870006,1.21651,1.2122,1.227,1623